In [1]:
import torch
from boardGPT.models import GameGPT
from transformers import AutoTokenizer

In [2]:
model, model_config = GameGPT.from_pretrained(repo_id="theartificialis/OthelloGPT-Synthetic-20m")
model.to("cuda")
model.eval()

GameGPT(
  (token_emb_hook): HookPoint()
  (pos_emb_hook): HookPoint()
  (pre_logits_hook): HookPoint()
  (transformer): ModuleDict(
    (wte): Embedding(61, 512)
    (wpe): Embedding(60, 512)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-7): 8 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (qk_hook): HookPoint()
          (v_hook): HookPoint()
          (c_attn): Linear(in_features=512, out_features=1536, bias=False)
          (c_proj): Linear(in_features=512, out_features=512, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=512, out_features=2048, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=2048, out_features=512, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    

In [14]:
print(f"block_size={model_config.block_size}")
print(f"vocab_size={model_config.vocab_size}")
print(f"n_layers={model_config.n_layers}")
print(f"n_heads={model_config.n_heads}")
print(f"n_embd={model_config.n_embd}")
print(f"dropout={model_config.dropout}")
print()

GPTConfig(block_size=60, vocab_size=61, n_layer=8, n_head=8, n_embd=512, dropout=0.0, n_latent_token=None, n_latent=None, bias=False)

In [3]:
tokenizer = AutoTokenizer.from_pretrained("theartificialis/OthelloGPT-Synthetic-20m", subfolder="tokenizer")

In [4]:
input_ids = tokenizer("a4 b3")['input_ids']
input_ids = torch.tensor(input_ids, dtype=torch.long)
input_ids = torch.unsqueeze(input_ids, dim=0)
input_ids = input_ids.to("cuda")
input_ids

tensor([[ 4, 11]], device='cuda:0')

In [5]:
x, logits, loss, objects = model(
    idx=input_ids,
    to_return=["residuals0"]
)

In [6]:
# x: derniers residuals (512 par token)
x.shape

torch.Size([1, 2, 512])

In [7]:
# logits, probs pour le dernier token de la séquence
logits.shape

torch.Size([1, 1, 61])

In [8]:
print(loss)

None


In [13]:
# Residual du bloc zéro
objects[0].shape

torch.Size([1, 2, 512])

In [10]:
model

GameGPT(
  (token_emb_hook): HookPoint()
  (pos_emb_hook): HookPoint()
  (pre_logits_hook): HookPoint()
  (transformer): ModuleDict(
    (wte): Embedding(61, 512)
    (wpe): Embedding(60, 512)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-7): 8 x Block(
        (ln_1): LayerNorm()
        (attn): CausalSelfAttention(
          (qk_hook): HookPoint()
          (v_hook): HookPoint()
          (c_attn): Linear(in_features=512, out_features=1536, bias=False)
          (c_proj): Linear(in_features=512, out_features=512, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm()
        (mlp): MLP(
          (c_fc): Linear(in_features=512, out_features=2048, bias=False)
          (gelu): GELU(approximate='none')
          (c_proj): Linear(in_features=2048, out_features=512, bias=False)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    